# Neural Network Prediction of Duffing Oscillator

**Team 19**: Vlad-Flavius Misăilă, Robert-Daniel Man, Sebastian-Adrian Mărginean

## Overview

The Duffing oscillator is a forced nonlinear oscillator that can exhibit both periodic and chaotic behavior:

$$
\frac{d^2x}{dt^2} + \delta\frac{dx}{dt} + \alpha x + \beta x^3 = \gamma\cos(\omega t)
$$

As a first-order system:
$$
\begin{align}
\frac{dx}{dt} &= y \\
\frac{dy}{dt} &= -\delta y - \alpha x - \beta x^3 + \gamma\cos(\omega t)
\end{align}
$$

**Parameters**:
- $\alpha = -1.0$: Linear stiffness (negative for double-well potential)
- $\beta = 1.0$: Nonlinear stiffness
- $\delta = 0.3$: Damping coefficient
- $\gamma = 0.5$: Forcing amplitude
- $\omega = 1.2$: Forcing frequency

**Objective**: Predict both periodic and chaotic regimes of the forced Duffing oscillator.

In [ ]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

import numpy as np
import matplotlib.pyplot as plt
import torch

from src.dynamical_systems import DuffingOscillator
from src.data_preparation import generate_trajectory, create_sequences
from src.neural_models import FeedForwardPredictor, LSTMPredictor, NeuralPredictor
from src.evaluation import (
    evaluate_prediction, plot_trajectory_2d, plot_time_series,
    plot_training_history, prediction_horizon_analysis
)

np.random.seed(42)
torch.manual_seed(42)
plt.rcParams['figure.dpi'] = 100
%matplotlib inline

print("✓ Setup complete!")

## 1. Explore Different Forcing Amplitudes

The Duffing oscillator's behavior depends critically on forcing amplitude $\gamma$.

In [ ]:
# Test different forcing amplitudes
gamma_values = [0.2, 0.37, 0.5, 1.0]
regimes = ['Periodic', 'Near Chaos', 'Chaotic', 'Strongly Forced']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, (gamma, regime) in enumerate(zip(gamma_values, regimes)):
    duffing = DuffingOscillator(alpha=-1.0, beta=1.0, delta=0.3, gamma=gamma, omega=1.2)
    t, traj = generate_trajectory(
        duffing,
        initial_state=np.array([0.1, 0.1]),
        t_span=(0, 100),
        dt=0.01
    )
    
    # Plot phase portrait (skip transient)
    skip = 2000  # Skip transient
    axes[idx].plot(traj[skip:, 0], traj[skip:, 1], 'b-', linewidth=0.5, alpha=0.7)
    axes[idx].set_xlabel('x (position)', fontsize=11)
    axes[idx].set_ylabel('y (velocity)', fontsize=11)
    axes[idx].set_title(f'{regime} (γ = {gamma})', fontsize=12)
    axes[idx].grid(True, alpha=0.3)
    axes[idx].axhline(0, color='k', linewidth=0.5)
    axes[idx].axvline(0, color='k', linewidth=0.5)

plt.tight_layout()
plt.suptitle('Duffing Oscillator: Effect of Forcing Amplitude γ', fontsize=14, y=1.02)
plt.show()

print("✓ Explored different regimes")

## 2. Focus on Chaotic Regime (γ = 0.5)

In [ ]:
# Generate data in chaotic regime
duffing = DuffingOscillator(alpha=-1.0, beta=1.0, delta=0.3, gamma=0.5, omega=1.2)
initial_state = np.array([0.1, 0.1])

t, trajectory = generate_trajectory(
    duffing,
    initial_state=initial_state,
    t_span=(0, 200),  # Long trajectory
    dt=0.01
)

print(f"✓ Generated {len(t)} time points")
print(f"✓ Trajectory shape: {trajectory.shape}")

In [ ]:
# Visualize chaotic attractor
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Time series
n_plot = 3000
axes[0].plot(t[:n_plot], trajectory[:n_plot, 0], 'b-', linewidth=1, label='Position (x)')
axes[0].plot(t[:n_plot], trajectory[:n_plot, 1], 'r-', linewidth=1, alpha=0.7, label='Velocity (y)')
axes[0].set_xlabel('Time', fontsize=12)
axes[0].set_ylabel('Value', fontsize=12)
axes[0].set_title('Duffing Time Series (Chaotic Regime)', fontsize=13)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Phase portrait (skip transient)
skip = 1000
axes[1].plot(trajectory[skip:, 0], trajectory[skip:, 1], 'b-', linewidth=0.3, alpha=0.7)
axes[1].set_xlabel('x (position)', fontsize=12)
axes[1].set_ylabel('y (velocity)', fontsize=12)
axes[1].set_title('Duffing Strange Attractor', fontsize=13)
axes[1].grid(True, alpha=0.3)
axes[1].axhline(0, color='k', linewidth=0.5)
axes[1].axvline(0, color='k', linewidth=0.5)

plt.tight_layout()
plt.show()

## 3. Prepare Data and Train Models

In [ ]:
# Remove initial transient
skip_transient = 5000
trajectory_stable = trajectory[skip_transient:]
t_stable = t[skip_transient:] - t[skip_transient]

print(f"After removing transient: {trajectory_stable.shape}")

In [ ]:
# Prepare sequences
WINDOW_SIZE = 50
X_train, y_train, X_test, y_test, scaler = create_sequences(
    trajectory_stable, window_size=WINDOW_SIZE, train_ratio=0.8, normalize=True
)

print(f"Training: X={X_train.shape}, y={y_train.shape}")
print(f"Testing: X={X_test.shape}, y={y_test.shape}")

In [ ]:
# Train Feed-Forward Network
print("\nTraining Feed-Forward Network...")
fnn = FeedForwardPredictor(WINDOW_SIZE * 2, [128, 64, 32], 2, dropout=0.1)
fnn_predictor = NeuralPredictor(fnn, learning_rate=0.001)
fnn_history = fnn_predictor.train(X_train, y_train, X_test, y_test, epochs=100, batch_size=64)

plot_training_history(fnn_history, 'FNN Training (Duffing)')

In [ ]:
# Train LSTM Network
print("\nTraining LSTM Network...")
lstm = LSTMPredictor(2, 64, 2, 2, dropout=0.1)
lstm_predictor = NeuralPredictor(lstm, learning_rate=0.001)
lstm_history = lstm_predictor.train(X_train, y_train, X_test, y_test, epochs=100, batch_size=64)

plot_training_history(lstm_history, 'LSTM Training (Duffing)')

## 4. Evaluate Models

In [ ]:
# One-step predictions
fnn_pred = fnn_predictor.predict(X_test)
lstm_pred = lstm_predictor.predict(X_test)

fnn_metrics = evaluate_prediction(y_test, fnn_pred)
lstm_metrics = evaluate_prediction(y_test, lstm_pred)

print("="*60)
print("ONE-STEP PREDICTION RESULTS (Duffing Oscillator - Chaotic)")
print("="*60)
print(f"\nFeed-Forward: RMSE={fnn_metrics['rmse']:.6f}, MAE={fnn_metrics['mae']:.6f}")
print(f"LSTM:         RMSE={lstm_metrics['rmse']:.6f}, MAE={lstm_metrics['mae']:.6f}")
print("="*60)

In [ ]:
# Visualize one-step predictions
y_test_orig = scaler.inverse_transform(y_test)
fnn_pred_orig = scaler.inverse_transform(fnn_pred)
lstm_pred_orig = scaler.inverse_transform(lstm_pred)

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# FNN
axes[0].plot(y_test_orig[:500, 0], y_test_orig[:500, 1], 'b-', linewidth=2, alpha=0.5, label='True')
axes[0].plot(fnn_pred_orig[:500, 0], fnn_pred_orig[:500, 1], 'r--', linewidth=1.5, alpha=0.7, label='FNN Pred')
axes[0].set_xlabel('x (position)', fontsize=12)
axes[0].set_ylabel('y (velocity)', fontsize=12)
axes[0].set_title('FNN One-Step Predictions', fontsize=13)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# LSTM
axes[1].plot(y_test_orig[:500, 0], y_test_orig[:500, 1], 'b-', linewidth=2, alpha=0.5, label='True')
axes[1].plot(lstm_pred_orig[:500, 0], lstm_pred_orig[:500, 1], 'r--', linewidth=1.5, alpha=0.7, label='LSTM Pred')
axes[1].set_xlabel('x (position)', fontsize=12)
axes[1].set_ylabel('y (velocity)', fontsize=12)
axes[1].set_title('LSTM One-Step Predictions', fontsize=13)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Multi-Step Prediction and Attractor Reconstruction

In [ ]:
# Long-term prediction
initial_window = X_test[100]
n_steps = 500
true_future = y_test[100:100 + n_steps]

fnn_future = fnn_predictor.iterative_predict(initial_window, n_steps)
lstm_future = lstm_predictor.iterative_predict(initial_window, n_steps)

# Inverse transform
true_orig = scaler.inverse_transform(true_future)
fnn_orig = scaler.inverse_transform(fnn_future)
lstm_orig = scaler.inverse_transform(lstm_future)

print(f"✓ Generated {n_steps}-step predictions")

In [ ]:
# Compare attractors
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# True
axes[0].plot(true_orig[:, 0], true_orig[:, 1], 'b-', linewidth=0.8, alpha=0.7)
axes[0].set_title('True Attractor', fontsize=13)
axes[0].set_xlabel('x (position)')
axes[0].set_ylabel('y (velocity)')
axes[0].grid(True, alpha=0.3)

# FNN
axes[1].plot(fnn_orig[:, 0], fnn_orig[:, 1], 'r-', linewidth=0.8, alpha=0.7)
axes[1].set_title('FNN Predicted Attractor', fontsize=13)
axes[1].set_xlabel('x (position)')
axes[1].set_ylabel('y (velocity)')
axes[1].grid(True, alpha=0.3)

# LSTM
axes[2].plot(lstm_orig[:, 0], lstm_orig[:, 1], 'g-', linewidth=0.8, alpha=0.7)
axes[2].set_title('LSTM Predicted Attractor', fontsize=13)
axes[2].set_xlabel('x (position)')
axes[2].set_ylabel('y (velocity)')
axes[2].grid(True, alpha=0.3)

plt.suptitle(f'{n_steps}-Step Predictions: Attractor Structure', fontsize=14)
plt.tight_layout()
plt.show()

## 6. Prediction Horizon Analysis

In [ ]:
# Analyze error growth
fnn_errors, steps = prediction_horizon_analysis(
    fnn_predictor, initial_window, true_future, max_steps=n_steps
)
lstm_errors, _ = prediction_horizon_analysis(
    lstm_predictor, initial_window, true_future, max_steps=n_steps
)

# Plot
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(steps, fnn_errors, 'b-', linewidth=2, label='FNN', alpha=0.7)
ax.plot(steps, lstm_errors, 'r-', linewidth=2, label='LSTM', alpha=0.7)
ax.set_xlabel('Prediction Horizon (steps)', fontsize=12)
ax.set_ylabel('RMSE', fontsize=12)
ax.set_title('Prediction Error Growth (Duffing Oscillator - Chaotic Regime)', fontsize=14)
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)
ax.set_yscale('log')
plt.tight_layout()
plt.show()

## 7. Periodic Regime Comparison

Test model performance on periodic regime (lower forcing amplitude).

In [ ]:
# Generate periodic regime data
duffing_periodic = DuffingOscillator(alpha=-1.0, beta=1.0, delta=0.3, gamma=0.2, omega=1.2)
t_p, traj_p = generate_trajectory(
    duffing_periodic,
    initial_state=np.array([0.1, 0.1]),
    t_span=(0, 100),
    dt=0.01
)

# Remove transient and prepare data
traj_p_stable = traj_p[5000:]
X_train_p, y_train_p, X_test_p, y_test_p, scaler_p = create_sequences(
    traj_p_stable, window_size=WINDOW_SIZE, train_ratio=0.8, normalize=True
)

# Train quick model
print("Training on periodic regime...")
lstm_periodic = NeuralPredictor(LSTMPredictor(2, 32, 2, 2), learning_rate=0.001)
_ = lstm_periodic.train(X_train_p, y_train_p, epochs=50, verbose=False)

# Predict
pred_periodic = lstm_periodic.predict(X_test_p)
metrics_periodic = evaluate_prediction(y_test_p, pred_periodic)

print(f"\nPeriodic regime RMSE: {metrics_periodic['rmse']:.6f}")
print("Note: Lower error due to regular, periodic behavior")

## 8. Summary

### Key Findings:

1. **Regime Dependence**:
   - Periodic regime: High predictability, low error
   - Chaotic regime: Limited prediction horizon, exponential error growth

2. **Model Comparison**:
   - LSTM outperforms FNN, especially for longer prediction horizons
   - Both models can capture short-term dynamics
   - Long-term attractor structure preservation is challenging

3. **Forcing Effects**:
   - Small γ → periodic orbits → easier to predict
   - Large γ → chaos → fundamental predictability limits

4. **Comparison with Other Systems**:
   - **vs Lorenz/Rössler**: Similar chaotic behavior and prediction limits
   - **vs Van der Pol**: More complex due to external forcing
   - **Non-autonomous**: Time-dependent forcing adds complexity

### Physical Insights:

- The Duffing oscillator models systems with cubic nonlinearity (e.g., buckled beams, magneto-elastic ribbons)
- Double-well potential ($\alpha < 0$) enables chaotic switching between wells
- Route to chaos occurs through period-doubling cascades

### Applications:

- Structural vibrations
- Energy harvesting systems
- Nonlinear optics
- Electrical circuits

In [ ]:
print("\n✓ Duffing oscillator analysis complete!")